In [1]:
from google.colab import drive

drive.mount('/gdrive')
gdrive_root = '/gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
batch_size = 100
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,), std=(0.5,))])
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transform,
                              download=True)
 
test_dataset = datasets.MNIST(root='./mnist_data/',
                             train=False,
                             transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

0it [00:00, ?it/s]

9920512it [00:01, 8793965.86it/s]                            


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 134993.81it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


1654784it [00:00, 2199258.08it/s]                            
0it [00:00, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


8192it [00:00, 51497.87it/s]            


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw
Processing...
Done!


In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(num_features=64)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels = 128, kernel_size=3, stride = 1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels = 64, kernel_size=3, stride = 2, padding=2)
        self.dropout1 = nn.Dropout(0.25)
        self.FC1 = nn.Linear(64*15*15, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.FC2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.relu(self.conv1(x)) 
        x = self.relu(self.conv2(x))
        x = self.dropout1(x)
        x = x.view(-1, 64*15*15)
        x = self.relu(self.FC1(x))
        x = self.dropout2(x)
        x = self.FC2(x)
        x = self.softmax(x)

        return x
    
    def get_output(self, x):
        in_size = x.size(0)
        x = self.relu(self.conv1(x)) 
        x = self.relu(self.conv2(x))
        x = x.view(-1, 64*15*15)
        x = self.relu(self.FC1(x))
        x = self.FC2(x)
        x = self.softmax(x)

        return x        

In [0]:
model = Net()
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

best_acc = 0.
ckpt_dir = os.path.join(gdrive_root, 'checkpoints')
if not os.path.exists(ckpt_dir):
  os.makedirs(ckpt_dir)

ckpt_path = os.path.join(ckpt_dir, 'modelC_ckpt_norm.pt')
# if os.path.exists(ckpt_path):
#   ckpt = torch.load(ckpt_path)
#   try:
#     model.load_state_dict(ckpt['model'])
#     optimizer.load_state_dict(ckpt['optimizer'])
#     best_acc = ckpt['best_acc']
#   except RuntimeError as e:
#     print('wrong checkpoint')
#   else:    
#     print('checkpoint is loaded !')
#     print('current best accuracy : %.2f' % best_acc)

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

def test(best_acc):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).item()
        
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    test_acc = correct / float(len(test_loader.dataset))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * test_acc))

    if test_acc > best_acc:
      best_acc = test_acc
      ckpt = {'model': model.state_dict(),
              'optimizer':optimizer.state_dict(),
              'best_acc':best_acc}
      torch.save(ckpt, ckpt_path)
      print('checkpoint is saved!')

In [6]:
for epoch in range(1, 10):
    train(epoch)
    test(best_acc)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303764
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.302285
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.302195
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.301548
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.300707
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.300475
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.299567
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.300010
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.300414
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.298909
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.298767
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.296349
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.296109
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.292308
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.298126
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.290891
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.292554
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.284077
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.276522
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.281646
Train Epoch: 1 [200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0169, Accuracy: 7881/10000 (79%)

checkpoint is saved!
Train Epoch: 2 [0/60000 (0%)]	Loss: 1.767553
Train Epoch: 2 [1000/60000 (2%)]	Loss: 1.739342
Train Epoch: 2 [2000/60000 (3%)]	Loss: 1.685534
Train Epoch: 2 [3000/60000 (5%)]	Loss: 1.733724
Train Epoch: 2 [4000/60000 (7%)]	Loss: 1.774789
Train Epoch: 2 [5000/60000 (8%)]	Loss: 1.668955
Train Epoch: 2 [6000/60000 (10%)]	Loss: 1.769354
Train Epoch: 2 [7000/60000 (12%)]	Loss: 1.687608
Train Epoch: 2 [8000/60000 (13%)]	Loss: 1.814411
Train Epoch: 2 [9000/60000 (15%)]	Loss: 1.711981
Train Epoch: 2 [10000/60000 (17%)]	Loss: 1.745189
Train Epoch: 2 [11000/60000 (18%)]	Loss: 1.664484
Train Epoch: 2 [12000/60000 (20%)]	Loss: 1.705067
Train Epoch: 2 [13000/60000 (22%)]	Loss: 1.726520
Train Epoch: 2 [14000/60000 (23%)]	Loss: 1.695350
Train Epoch: 2 [15000/60000 (25%)]	Loss: 1.800336
Train Epoch: 2 [16000/60000 (27%)]	Loss: 1.712779
Train Epoch: 2 [17000/60000 (28%)]	Loss: 1.706469
Train Epoch: 2 [18000/60000 (30%)]	Lo